# import

In [4]:
from astropy.io import fits
import numpy as np
import torch
from torch import nn, optim
from torch.utils import data
from astropy.nddata.utils import Cutout2D
import glob, os
from esutil import wcsutil
from astropy.table import Table
import pandas as pd
import fitsio
import seaborn as sns
import matplotlib.pyplot as plt
import collections
from random import randint
import math
import sys
from astropy.visualization import hist
from torch.utils.data import DataLoader
from astroML.datasets import fetch_imaging_sample, fetch_sdss_S82standards
import torch.nn.functional as F
import skimage
from tensorboardX import SummaryWriter

# config

In [5]:
import argparse

# ----------------------------------------
# Global variables within this script
arg_lists = []
parser = argparse.ArgumentParser()


def str2bool(v):
    return v.lower() in ("true", "1")


def add_argument_group(name):
    arg = parser.add_argument_group(name)
    arg_lists.append(arg)
    return arg


# ----------------------------------------
# Arguments for the main program
main_arg = add_argument_group("Main")


main_arg.add_argument("--mode", type=str,
                      default="train",
                      choices=["train", "test"],
                      help="Run mode")

# ----------------------------------------
# Arguments for training
train_arg = add_argument_group("Training")


train_arg.add_argument("--data_dir", type=str,
                       default="/Users/kwang/Downloads/cifar-10-batches-py",
                       help="Directory with CIFAR10 data")

train_arg.add_argument("--learning_rate", type=float,
                       default=1e-3,
                       help="Learning rate (gradient step size)")

train_arg.add_argument("--batch_size", type=int,
                       default=1,
                       help="Size of each training batch")

train_arg.add_argument("--num_epoch", type=int,
                       default=100,
                       help="Number of epochs to train")

train_arg.add_argument("--val_intv", type=int,
                       default=1000,
                       help="Validation interval")

train_arg.add_argument("--rep_intv", type=int,
                       default=150,
                       help="Report interval")

train_arg.add_argument("--log_dir", type=str,
                       default="/home/yufeng/projects/rrg-kyi/yufeng/image_classification/unet/data/loss_plot",
                       help="Directory to save logs and current model")

train_arg.add_argument("--save_dir", type=str,
                       default="./save",
                       help="Directory to save the best model")

train_arg.add_argument("--resume", type=str2bool,
                       default=True,
                       help="Whether to resume training from existing checkpoint")
# ----------------------------------------
# Arguments for model
model_arg = add_argument_group("Model")

model_arg.add_argument("--feature_type", type=str,
                       default="hog",
                       choices=["hog", "h_histogram", "rgb"],
                       help="Type of feature to be used")

model_arg.add_argument("--loss_type", type=str,
                       default="cross_entropy",
                       choices=["cross_entropy", "svm"],
                       help="Type of data loss to be used")

model_arg.add_argument("--normalize", type=str2bool,
                       default=True,
                       help="Whether to normalize with mean/std or not")

model_arg.add_argument("--l2_reg", type=float,
                       default=1e-3,
                       help="L2 Regularization strength")

model_arg.add_argument("--num_unit", type=int,
                       default=64,
                       help="Number of neurons in the hidden layer")

model_arg.add_argument("--num_hidden", type=int,
                       default=3,
                       help="Number of hidden layers")

model_arg.add_argument("--num_class", type=int,
                       default=10,
                       help="Number of classes in the dataset")

model_arg.add_argument("--activ_type", type=str,
                       default="relu",
                       choices=["relu", "tanh"],
                       help="Activation type")


def get_config():
    config, unparsed = parser.parse_known_args()

    return config, unparsed


def print_usage():
    parser.print_usage()

In [6]:
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, input):
        return self.conv(input)


class MyUnet(nn.Module):
    def __init__(self,in_ch,out_ch):
        super(MyUnet, self).__init__()

        self.conv1 = DoubleConv(in_ch, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.conv4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)
        self.conv5 = DoubleConv(512, 1024)
        self.up6 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.conv6 = DoubleConv(1024, 512)
        self.up7 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.conv7 = DoubleConv(512, 256)
        self.up8 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.conv8 = DoubleConv(256, 128)
        self.up9 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.conv9 = DoubleConv(128, 64)
        self.conv10 = nn.Conv2d(64,out_ch, 1)

    def forward(self,x):
        c1=self.conv1(x)
        p1=self.pool1(c1)
        c2=self.conv2(p1)
        p2=self.pool2(c2)
        c3=self.conv3(p2)
        p3=self.pool3(c3)
        c4=self.conv4(p3)
        p4=self.pool4(c4)
        c5=self.conv5(p4)
        up_6= self.up6(c5)
        merge6 = torch.cat([up_6, c4], dim=1)
        c6=self.conv6(merge6)
        up_7=self.up7(c6)
        merge7 = torch.cat([up_7, c3], dim=1)
        c7=self.conv7(merge7)
        up_8=self.up8(c7)
        merge8 = torch.cat([up_8, c2], dim=1)
        c8=self.conv8(merge8)
        up_9=self.up9(c8)
        merge9=torch.cat([up_9,c1],dim=1)
        c9=self.conv9(merge9)
        c10=self.conv10(c9)
        #out = nn.Sigmoid()(c10)
        return c10

# DataWrap

In [7]:
class PitcairnDataset(data.Dataset):
    def __init__(self, config, mode):
        # mode is "train", "validation" or "test"
        self.config = config
        print("Loading PitcairnDataset")
        data, label = load_data()
        self.data = data
        self.label = label
        print("done.")
    
    def __len__(self):
        return (self.data.shape[0])
    
    def __getitem__(self, index):
        data_cur = self.data[index]
        # data
        data_cur = torch.from_numpy(data_cur.astype(np.float32))
        # label
        label_cur = self.label[index]
        return data_cur, label_cur

In [8]:
def load_data():
    # read input images
    os.chdir("/home/yufeng/scratch/cfis")
    data = []
    label = []
    x_slice = 264
    y_slice = 258
    x_num = 8
    y_num = 18
    
    for file in glob.glob("*.fits.fz"):
        file_id = file[:-8]
        print(file_id)
        data_image = fits.open(file, memmap=True)
        path = '/home/yufeng/projects/rrg-kyi/yufeng/image_classification/data/label_image/'+file_id+'.fits.gz'
        label_image = fits.open(path)
        for k in range(1, 41):
            label_image_blur = skimage.filters.gaussian(np.array(label_image[k].data), sigma=0.4, truncate=3.5, multichannel=True)
            for i in range(x_num):
                for j in range(y_num):
                    x = np.array(data_image[k].data.T[i*x_slice:(i+1)*x_slice, j*y_slice:(j+1)*y_slice])
                    y = np.array(label_image_blur[i*x_slice:(i+1)*x_slice, j*y_slice:(j+1)*y_slice,:])
                    if np.any(y>0.5):
                        data += [x]
                        label += [y]
            break
                
    data = np.array(data)
    data = np.transpose(data.reshape(data.shape[0], data.shape[1], data.shape[2], 1), (0, 3, 1, 2))
    label = np.array(label)
    label = np.transpose(label, (0, 3, 1, 2))
    data = data[:, :, :256, :256]
    label = label[:, :, :256, :256]
    print(np.array(data).shape)
    print(np.array(label).shape)
    return data, label

In [43]:
def load_data():
    # read input images
    os.chdir("/home/yufeng/scratch/cfis")
    data = []
    label = []
    x_slice = 264
    y_slice = 258
    x_num = 8
    y_num = 18
    # cropping images based on object
    
    for file in glob.glob("*.fits.fz"):
        file_id = file[:-8]
        print(file_id)
        data_image = fits.open(file, memmap=True)
        path = '/home/yufeng/projects/rrg-kyi/yufeng/image_classification/data/label_image/'+file_id+'.fits.gz'
        label_image = fits.open(path)
        for k in range(1, 41):
            x = data_image[k].data
            y = label_image[k].data
            label_pixel = y != 0.25
            print(y[label_pixel])
    data = np.array(data)
    data = np.transpose(data.reshape(data.shape[0], data.shape[1], data.shape[2], 1), (0, 3, 1, 2))
    label = np.array(label)
    label = np.transpose(label, (0, 3, 1, 2))
    data = data[:, :, :256, :256]
    label = label[:, :, :256, :256]
    print(np.array(data).shape)
    print(np.array(label).shape)
    return data, label

# solution

In [13]:
def train(config):
    train_data = PitcairnDataset(config, mode="train")
    inc = 1
    outc = 4
    model = MyUnet(inc, outc)
    
    if torch.cuda.is_available():
        model = model.cuda()
    
    tr_data_loader = DataLoader(
        dataset=train_data,
        batch_size=config.batch_size,
        num_workers=2,
        shuffle=True)
    
    model.train()
    
    loss = nn.PoissonNLLLoss()
    
    optimizer = optim.Adam(model.parameters(), lr = config.learning_rate)
    
    iter_idx = -1
    tr_writer = SummaryWriter(
        log_dir=os.path.join(config.log_dir, "train"))
    va_writer = SummaryWriter(
        log_dir=os.path.join(config.log_dir, "valid"))
    
    for epoch in range(config.num_epoch):
        print(epoch)
        prefix = "Training Epoch {:3d}: ".format(epoch)
        for data in tr_data_loader:
            iter_idx += 1
            x, y = data
            
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
            
            logits = model.forward(x)
            
            m = nn.Softmax2d()
            logits = m(logits)
            logits = torch.log(logits)
            
            temp_x = logits.detach().cpu().numpy()
            temp_y = y.detach().cpu().numpy()
            
            label_pixel = temp_y != 0.25

            #output_loss = loss(logits, y)
            output_loss = loss(torch.from_numpy(temp_x[label_pixel]).requires_grad_(), torch.from_numpy(temp_y[label_pixel]).requires_grad_())
            output_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            tr_writer.add_scalar("loss", output_loss, global_step=iter_idx)
            if iter_idx % config.rep_intv == 0:
                print("training loss: " + str(output_loss))
                print('prediction: ')
                print(np.exp(temp_x[label_pixel]))
                print('label: ')
                print(temp_y[label_pixel])
                                

# main

In [15]:
def main(config):
    """The main function."""

    if config.mode == "train":
        train(config)
    elif config.mode == "test":
        test(config)
    else:
        raise ValueError("Unknown run mode \"{}\"".format(config.mode))


if __name__ == "__main__":

    # ----------------------------------------
    # Parse configuration
    config, unparsed = get_config()
    # If we have unparsed arguments, print usage and exit
    if len(unparsed) > 0:
        print_usage()
        exit(1)

    main(config)

usage: ipykernel_launcher.py [-h] [--mode {train,test}] [--data_dir DATA_DIR]
                             [--learning_rate LEARNING_RATE]
                             [--batch_size BATCH_SIZE] [--num_epoch NUM_EPOCH]
                             [--val_intv VAL_INTV] [--rep_intv REP_INTV]
                             [--log_dir LOG_DIR] [--save_dir SAVE_DIR]
                             [--resume RESUME]
                             [--feature_type {hog,h_histogram,rgb}]
                             [--loss_type {cross_entropy,svm}]
                             [--normalize NORMALIZE] [--l2_reg L2_REG]
                             [--num_unit NUM_UNIT] [--num_hidden NUM_HIDDEN]
                             [--num_class NUM_CLASS]
                             [--activ_type {relu,tanh}]
Loading PitcairnDataset
2301818p
2301538p
2301539p
(73, 1, 256, 256)
(73, 4, 256, 256)
done.
0
training loss: tensor(0.6023, dtype=torch.float64, grad_fn=<MeanBackward0>)
prediction: 
[0.18348409 0.21227

1


2
training loss: tensor(0.6023, dtype=torch.float64, grad_fn=<MeanBackward0>)
prediction: 
[0.18348409 0.21227695 0.2611119  0.20448133 0.23948841 0.2623309
 0.25210065 0.29580387 0.27577707 0.33488563 0.21057457 0.3200012
 0.36378223 0.22564223 0.24489273 0.26430416 0.24435075 0.27597505
 0.19922678 0.32439974 0.198127   0.20913653 0.18255188 0.28023103
 0.19963448 0.22123313 0.18941897 0.2824035  0.2527487  0.22075996
 0.22259992 0.35231754 0.21254537 0.2839607  0.23861222 0.25882894]
label: 
[0.2495922  0.24071862 0.2495922  0.24071862 0.03875671 0.24071862
 0.2495922  0.24071862 0.2495922  0.25122339 0.27784415 0.25122339
 0.27784415 0.88372987 0.27784415 0.25122339 0.27784415 0.25122339
 0.2495922  0.24071862 0.2495922  0.24071862 0.03875671 0.24071862
 0.2495922  0.24071862 0.2495922  0.2495922  0.24071862 0.2495922
 0.24071862 0.03875671 0.24071862 0.2495922  0.24071862 0.2495922 ]


3


4
training loss: tensor(0.6056, dtype=torch.float64, grad_fn=<MeanBackward0>)
prediction: 
[0.2584194  0.20465508 0.2576853  0.18521433 0.27033925 0.2531475
 0.23699766 0.3095699  0.20003395 0.28789687 0.30849284 0.2041372
 0.30194965 0.23822904 0.26198292 0.2632125  0.28657407 0.31386012
 0.19309022 0.18822533 0.28363156 0.2077205  0.21462786 0.18890488
 0.22547685 0.14561282 0.18200602 0.2605935  0.29862666 0.2545459
 0.3051155  0.27680388 0.2959647  0.27431303 0.2582432  0.30409995]
label: 
[0.2495922  0.24071862 0.2495922  0.24071862 0.03875671 0.24071862
 0.2495922  0.24071862 0.2495922  0.2495922  0.24071862 0.2495922
 0.24071862 0.03875671 0.24071862 0.2495922  0.24071862 0.2495922
 0.25122339 0.27784415 0.25122339 0.27784415 0.88372987 0.27784415
 0.25122339 0.27784415 0.25122339 0.2495922  0.24071862 0.2495922
 0.24071862 0.03875671 0.24071862 0.2495922  0.24071862 0.2495922 ]


...

training loss: tensor(0.6003, dtype=torch.float64, grad_fn=<MeanBackward0>)
prediction: 
[0.24015285 0.28496674 0.22980489 0.26252156 0.24957594 0.22837187
 0.27533513 0.2599957  0.24426644 0.26535508 0.24125165 0.23123144
 0.26182535 0.26148045 0.29266682 0.21286795 0.2670623  0.2399134
 0.22385918 0.21973456 0.26110214 0.21729663 0.22910932 0.1991156
 0.25294885 0.20849048 0.25340143 0.27063292 0.254047   0.27786154
 0.25835642 0.25983435 0.27984565 0.2588481  0.26445144 0.26241875]
label: 
[0.2495922  0.24071862 0.2495922  0.24071862 0.03875671 0.24071862
 0.2495922  0.24071862 0.2495922  0.2495922  0.24071862 0.2495922
 0.24071862 0.03875671 0.24071862 0.2495922  0.24071862 0.2495922
 0.25122339 0.27784415 0.25122339 0.27784415 0.88372987 0.27784415
 0.25122339 0.27784415 0.25122339 0.2495922  0.24071862 0.2495922
 0.24071862 0.03875671 0.24071862 0.2495922  0.24071862 0.2495922 ]
95


96


training loss: tensor(0.6089, dtype=torch.float64, grad_fn=<MeanBackward0>)
prediction: 
[0.4463287  0.33909822 0.31883693 0.3840462  0.26016873 0.41578192
 0.36660016 0.2890388  0.28321722 0.19360217 0.24043855 0.1755523
 0.21500866 0.3020138  0.29885107 0.23655793 0.35963622 0.29320446
 0.23440152 0.23351705 0.27358732 0.2230225  0.25608364 0.1579124
 0.24850884 0.22632444 0.22373417 0.12566759 0.18694612 0.23202349
 0.17792265 0.18173388 0.12745455 0.1483331  0.12500057 0.1998442 ]
label: 
[0.2495922  0.24071862 0.2495922  0.24071862 0.03875671 0.24071862
 0.2495922  0.24071862 0.2495922  0.2495922  0.24071862 0.2495922
 0.24071862 0.03875671 0.24071862 0.2495922  0.24071862 0.2495922
 0.25122339 0.27784415 0.25122339 0.27784415 0.88372987 0.27784415
 0.25122339 0.27784415 0.25122339 0.2495922  0.24071862 0.2495922
 0.24071862 0.03875671 0.24071862 0.2495922  0.24071862 0.2495922 ]
97


98


training loss: tensor(0.6036, dtype=torch.float64, grad_fn=<MeanBackward0>)
prediction: 
[0.24327268 0.27953646 0.21838345 0.27326742 0.17760102 0.26159936
 0.27169093 0.19821845 0.27847812 0.30827978 0.3530945  0.35472396
 0.23202765 0.31331515 0.21657786 0.18308899 0.27863446 0.23860985
 0.27529028 0.15184762 0.33806214 0.33098438 0.29997975 0.21503286
 0.31435117 0.2915699  0.21706636 0.22706349 0.17686255 0.25643662
 0.2736541  0.23276718 0.1738861  0.45189843 0.16479069 0.35092857
 0.23454466 0.22373278 0.2181102  0.20459163 0.25479952 0.2909142
 0.18070163 0.1610233  0.24427159 0.27416322 0.11171406 0.18954839
 0.2535418  0.26099217 0.30445608 0.17414157 0.2171952  0.17579702
 0.24689232 0.3448831  0.22544438 0.19115658 0.26761967 0.23245358
 0.23325622 0.3491884  0.26018384 0.19049345 0.35832894 0.1992911
 0.2407764  0.19292547 0.30507994 0.19087093 0.3393797  0.23466456]
label: 
[0.2495922  0.24071862 0.2495922  0.24071862 0.03875671 0.24071862
 0.2495922  0.24071862 0.2495922 